# Building and post-processing problem set 1 with FloPy

## PS1A

The model domain will be discretized into 3 layers, 21 rows, and 20 columns. A constant value of 500 ft will be specified for `delr` and `delc`. The top (`TOP`) of the model should be set to 400 ft and the bottom of the three layers (`BOTM`) should be set to 220 ft, 200 ft, and 0 ft, respectively. The model has one steady-state stress period. 

MODFLOW does not require that input data be provided in specific units (for example, SI units) instead it only requires that consistent units be used. As a result all input data should be specified with a length unit of feet and a time unit of days.

In [ ]:
import pathlib as pl
import platform

import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

Before creating any of the MODFLOW 6 FloPy objects you should define the simulation workspace (`ws`) where the model files are and the simulation name (`name`).

In [ ]:
ws = pl.Path("PS1a")
name = "ps1"

## Build the model

Create a simulation object, a temporal discretization object, and a iterative model solution object using `flopy.mf6.MFSimulation()`, `flopy.mf6.ModflowTdis()`, and `flopy.mf6.ModflowIms()`, respectively. Set the `sim_name` to `name` and `sim_ws` to `ws` in the simulation object. Use default values for all temporal discretization and iterative model solution variables. Make sure to include the simulation object (`sim`) as the first variable in the temporal discretization and iterative model solution objects. 

**NOTE:** Variables with default values (for example, `time_units=None` in `flopy.mf6.ModflowTdis()`).

In [ ]:
# create simulation (sim = flopy.mf6.MFSimulation())
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws)

# create tdis package (tdis = flopy.mf6.ModflowTdis(sim))
tdis = flopy.mf6.ModflowTdis(sim)

# create iterative model solution (ims = flopy.mf6.ModflowIms(sim))
ims = flopy.mf6.ModflowIms(sim)

Create the groundwater flow model object (`gwf`) using `flopy.mf6.ModflowGwf()`. Make sure to include the simulation object (`sim`) as the first variable in the groundwater flow model object and set `modelname` to `name`. Use `Shift-Tab` to see the optional variables that can be specified.

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True, print_flows=True)

Create the discretization package using `flopy.mf6.ModflowGwfdis()`. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `DIS` package (`flopy.mf6.ModflowGwfdis()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-dis.html).

FloPy can accomodate all of the options for specifying array data for a model. `CONSTANT` values for a variable can be specified by using a `float` or `int` python variable (as is done below for `DELR`, `DELC`, and `TOP`). `LAYERED` data can be specified by using a list or `CONSTANT` values for each layer (as is done below for `BOTM` data) or a list of numpy arrays or lists. Three-Dimensional data can be specified using a three-dimensional numpy array (with a shape of `(nlay, nrow, ncol)`) for this example. More information on how to specify array data can be found in the [FloPy ReadTheDocs](https://flopy.readthedocs.io/en/latest/Notebooks/mf6_data_tutorial07.html#MODFLOW-6:-Working-with-MODFLOW-Grid-Array-Data). 

In [ ]:
# dis data
nlay, nrow, ncol = 3, 21, 20
delr = delc = 500.
top = 400
botm = [220, 200, 0]

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm)

### Create the initial conditions (IC) package

Create the initial conditions package (`IC`) using `flopy.mf6.ModflowGwfic()` and set the initial head (`strt`) to 320. Default values can be used for the rest of the initial conditions package input. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `IC` package (`flopy.mf6.ModflowGwfic()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-ic.html).

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=420)

### Create the node property flow (NPF) package

The hydraulic properties for the model are defined in the image above and are specified in the node property flow package (`NPF`) using `flopy.mf6.ModflowGwfnpf()`. The first layer should be convertible (unconfined) and the remaining two layers will be non-convertible so `icelltype` should be `[1, 0, 0]`. The horizontal (`k`) and vertical (`k33`) conductivity should also be assigned as python lists (`[,,]`) and the values shown in the image above. The variable `save_specific_discharge` should be set to `True` so that specific discharge data are saved to the cell-by-cell file and can be used to plot discharge. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `NPF` package (`flopy.mf6.ModflowGwfic()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-npf.html).

In [ ]:
icelltype = [1, 0, 0]
k = [50, 0.01, 200]
k33 = [10, 0.01, 20]

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, icelltype=icelltype, k=k, k33=k33)

## Create the river chd

The river is located in layer 1 and column 20 in every row in the model. The river stage stage is 320. Use the `flopy.mf6.ModflowGwfchd()` method to specify well data for the river package (`RIV`). Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `RIV` package (`flopy.mf6.ModflowGwfchd()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-chd.html).

An example of a `stress_period_data` tuple for the `CHD` package is

```python
# (layer, row, column, head)
(0, 0, 0, 320.)
```

**HINT**: list comprehension is an easy way to create a constant head cell in every row in column 20 of the model.

**NOTE:** Remember the `pname="river"` tip to prevent creating multiple versions of the river package if you rerun the cell.


In [ ]:
riv_spd = [(0, i, ncol - 1, 320) for i in range(nrow)]

In [ ]:
riv_spd

In [ ]:
riv_chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=riv_spd, pname="river")

## Create the canal chd

The canal is located in layer 1 and column 1 in every row in the model. The canal stage stage is 330. Use the `flopy.mf6.ModflowGwfchd()` method to specify well data for the river package (`RIV`). Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `RIV` package (`flopy.mf6.ModflowGwfchd()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-chd.html).

An example of a `stress_period_data` tuple for the `CHD` package is

```python
# (layer, row, column, head)
(0, 0, 0, 330.)
```

**HINT**: list comprehension is an easy way to create a constant head cell in every row in column 1 of the model.

**NOTE:** Remember the `pname="canal"` tip to prevent creating multiple versions of the chd canal package if you rerun the cell.


In [ ]:
canal_spd = [(0, i, 0, 330) for i in range(nrow)]

In [ ]:
canal_spd

In [ ]:
canal_chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=canal_spd, pname="canal")

### Build output control

Define the output control package (`OC`) for the model using the `flopy.mf6.ModflowGwfoc()` method to `[('HEAD', 'ALL'), ('BUDGET', 'ALL')]` to save the head and flow for the model. Also the head (`head_filerecord`) and cell-by-cell flow (`budget_filerecord`) files should be set to `f"{name}.hds"` and `f"{name}.cbc"`, respectively. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `OC` package (`flopy.mf6.ModflowGwfoc()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-oc.html).

In [ ]:
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{name}.hds",
    budget_filerecord=f"{name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

### Write the model files and run the model

Write the MODFLOW 6 model files using `sim.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.write_simulation()`.

In [ ]:
sim.write_simulation()

Run the model using `sim.run_simulation()`, which will run the MODFLOW 6 executable installed in the Miniforge class environment (`pyclass`) and the MODFLOW 6 model files created with `.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.run_simulation()`.

In [ ]:
sim.run_simulation()

## Post-process the results


### Exercise 1
Load the listing file to see the inflows and outflows

In [ ]:
lst = gwf.output.list()

In [ ]:
ds = lst.get_dataframes()[1]

Load the heads and face flows from the hds file. The head file can be loaded with the `gwf.output.head().get_data()` method. Name the heads data `hds`.

In [ ]:
hds = gwf.output.head().get_data()

In [ ]:
hds[0, 0, :]

In [ ]:
hds[1, 0, :]

### Canal flux

In [ ]:
bmean = hds[0, 0, 0:2].mean() - botm[0]
bmean

In [ ]:
C = k[0] * bmean * delc / delr
C

In [ ]:
Q = C * (hds[0, 0, 0] - hds[0, 0, 1])
Q

In [ ]:
C12 = k33[1] * delr * delc / 10.
C12

In [ ]:
Q12 = C12 * (hds[0, 0, 0] - hds[1, 0, 0])
Q12

In [ ]:
21 * (Q + Q12)

### River flux

In [ ]:
bmean = hds[0, 0, 18:20].mean() - botm[0]
bmean

In [ ]:
C = k[0] * bmean * delc / delr
C

In [ ]:
Q = C * (hds[0, 0, 18] - hds[0, 0, 19])
Q

In [ ]:
C12 = k33[1] * delr * delc / 10.
C12

In [ ]:
Q12 = C12 * (hds[1, 0, -1] - hds[0, 0, -1])
Q12

In [ ]:
21 * (Q + Q12)

### Exercise 2

Load the budget flows from the cbc file. The cell-by-cell file can be loaded with the `gwf.output.budget().get_data()` method. 

In [ ]:
hds = gwf.output.head().get_data()

The unique records in the cell budget file can be determined using `.headers[["text", "imeth"]].drop_duplicates()`.

In [ ]:
gwf.output.budget().headers[["text", "imeth"]].drop_duplicates()

Retrieve the `'DATA-SPDIS'` data type from the cell-by-cell file. Name the specific discharge data `spd`.

Cell-by-cell data is returned as a list so access the data by using `spd = gwf.output.budget().get_data(text="DATA-SPDIS")[0]`.

In [ ]:
spd = gwf.output.budget().get_data(text="DATA-SPDIS")[0]

Plot the results using `flopy.plot.PlotMapView()`. The head results can be plotted using the `.plot_array()` method. The discharge results can be plotted using the `plot_specific_discharge()` method. Boundary conditions can be plotted using the `.plot_bc()` method.

In [ ]:
xs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 10})
cbv = xs.plot_array(hds, head=hds)
q = xs.plot_vector(spd["qx"], spd["qy"], spd["qz"], normalize=True)
xs.plot_bc("CHD")
xs.plot_bc("CHD")
xs.plot_grid(lw=0.5, color="black")

### Exercise 3

In [ ]:
hd = hds[:, 0, 10] - hds[:, 0, 9]
hd

In [ ]:
b = [hds[0, 0, 9:11].mean() - botm[0], botm[0] - botm[1], botm[1] - botm[2]]
b

In [ ]:
Q = hd * b * k
Q

In [ ]:
Q * 21

### Exercise 4

In [ ]:
C23 = k33[1] * delr * delc / 10.

In [ ]:
hd = hds[1, 0, :] - hds[2, 0, :]

In [ ]:
Q23 = C23 * hd
Q23

In [ ]:
Qup = 21 * Q23[:10].sum()
Qup

In [ ]:
Qdown = 21 * Q23[10:].sum()
Qdown

## PS1B

Double the Hydraulic Conductivity

In [ ]:
ws = pl.Path("PS1b")

In [ ]:
sim.set_sim_path(ws)

In [ ]:
k = [100, 0.02, 400]
k33 = [20, 0.02, 40]

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, icelltype=icelltype, k=k, k33=k33)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### How much water is leaving canal

In [ ]:
gwf.output.list().get_dataframes()[1]

In [ ]:
# PS1a water budget
ds

### How are heads affected

In [ ]:
hds1b = gwf.output.head().get_data()

In [ ]:
hds1b[0, 0, :]

In [ ]:
# PS1a kayer 1 heads
hds[0, 0, :]

In [ ]:
hds1b[1, 0, :]

In [ ]:
# PS1a layer 2 heads
hds[1, 0, :]

## PS1C

Change the horizontal and vertical conductivity of the silt layer

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="PS1a")

In [ ]:
ws = pl.Path("PS1c")
sim.set_sim_path(ws)

In [ ]:
gwf = sim.get_model()

In [ ]:
k = [50, 0.0001, 200]
k33 = [10, 0.0001, 20]

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, icelltype=icelltype, k=k, k33=k33)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### How are heads affected

In [ ]:
hds1c = gwf.output.head().get_data()

In [ ]:
hds1c[0, 0, :]

In [ ]:
# PS1a layer 1 heads
hds[0, 0, :]

In [ ]:
hds1c[1, 0, :]

In [ ]:
# PS1a layer 2 heads
hds[1, 0, :]

### Calculate the rate of flow between column 10 and 11

In [ ]:
hd1c = hds1c[:, 0, 10] - hds1c[:, 0, 9]
hd1c

In [ ]:
b1c = [hds1c[0, 0, 9:11].mean() - botm[0], botm[0] - botm[1], botm[1] - botm[2]]
b1c

In [ ]:
Q1c = hd1c * b1c * k
Q1c

In [ ]:
# PS1a results
Q

## PS1D

Remove the silt in the middle of the domain

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="PS1a")

In [ ]:
ws = pl.Path("PS1d")
sim.set_sim_path(ws)

In [ ]:
gwf = sim.get_model()

In [ ]:
kl2 = np.full((nrow, ncol), 0.0001)
k33l2 = np.full((nrow, ncol), 0.0001)
kl2[8:13, 4:15] = 50
k33l2[8:13, 4:15] = 10

In [ ]:
plt.imshow(kl2)

In [ ]:
k1d = [50, kl2, 200]
k331d = [10, k33l2, 20]

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, icelltype=icelltype, k=k1d, k33=k331d)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Summarize the effect on the groundwater flow pattern

In [ ]:
hds1d = gwf.output.head().get_data()

In [ ]:
spd1d = gwf.output.budget().get_data(text="DATA-SPDIS")[0]

In [ ]:
qx1d, qy1d, qz1d = flopy.utils.postprocessing.get_specific_discharge(spd1d, gwf)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, )
cbv = mm.plot_array(hds1d)
q = mm.plot_vector(qx1d, qy1d, normalize=True)
mm.plot_bc("CHD")
mm.plot_bc("CHD")
mm.plot_grid(lw=0.5, color="black")

In [ ]:
# PS1a results
mm = flopy.plot.PlotMapView(model=gwf, )
cbv = mm.plot_array(hds)
q = mm.plot_vector(spd["qx"], spd["qy"], normalize=True)
mm.plot_bc("CHD")
mm.plot_bc("CHD")
mm.plot_grid(lw=0.5, color="black")

## PS1E

Using idomain to make the hole in the silt.

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="PS1a")

In [ ]:
ws = pl.Path("PS1e")
sim.set_sim_path(ws)

In [ ]:
gwf = sim.get_model()

In [ ]:
idomainl2 = np.full((nrow, ncol), 1, dtype=int)
idomainl2[8:13, 4:15] = -1

In [ ]:
botml1 = np.full((nrow, ncol), 220.)
botml1[8:13, 4:15] = 200.

In [ ]:
idomain1e = [1, idomainl2, 1]
botm1e = [botml1, 200., 0]

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm1e, idomain=idomain1e)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Summarize the effect on the groundwater flow pattern

In [ ]:
hds1e = gwf.output.head().get_data()

In [ ]:
hds1e[0].min(), hds1e[0].max()

In [ ]:
spd1e = gwf.output.budget().get_data(text="DATA-SPDIS")[0]

In [ ]:
qx1e, qy1e, qz1e = flopy.utils.postprocessing.get_specific_discharge(spd1e, gwf)

In [ ]:
levels = np.arange(320, 331, 2)

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, )
cbv = mm.plot_array(hds1e)
mm.contour_array(hds1e, colors="white", levels=levels)
q = mm.plot_vector(qx1e, qy1e, normalize=True)
mm.plot_bc("CHD")
mm.plot_bc("CHD")
mm.plot_grid(lw=0.5, color="black")

In [ ]:
# PS1d results
mm = flopy.plot.PlotMapView(model=gwf, )
cbv = mm.plot_array(hds1d)
mm.contour_array(hds1d, colors="white", levels=levels)
q = mm.plot_vector(qx1d, qy1d, normalize=True)
mm.plot_bc("CHD")
mm.plot_bc("CHD")
mm.plot_grid(lw=0.5, color="black")